# Parctical example. Audiobooks

##### Problem

# Creating methods that will batch the data

In [102]:
import numpy as np
class Audiobooks_Data_Reader():
    #Dataset is a mandatory argument. batch_size is not, and defaults to None. 
    def __init__(self, dataset, batch_size = None):
        
        # if called with 'train' and '5', then it will load Audiobooks_data_train.npz with a batch_size of 5.
        npz = np.load('Audiobooks_data_{0}.npz'.format(dataset))
        
        # Two variables for inputs and targets
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        
        # Set the batch size to the full input size if not passed (for validation and test)
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        
        # Two forward slashes rounds the answer down to the nearest whole number
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
        
    #Method that loads the next batch
    def __next__(self):
        
        # Stop if we've exhausted the dataset
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
        
        # Slice the dataset into batches - the next function loads them one after another
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        
        # one hot encode the targets - in this case they are zeros and ones anyway, but here's the code
        classes_num = 2
#         j = np.arange(targets_batch.shape[0], dtype= np.int)
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[100:10] = 1
       # targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        return inputs_batch, targets_one_hot
    
    def __iter__(self):
        return self


# Creating the Machine Learning Algorithm


### import the relevant libraries .

In [103]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow.compat.v2 as tff
from datetime import datetime as dt

In [104]:
input_size = 10
output_size = 2
hidden_layer_size = 50

# Clear from memory the variables from previous runs. Variables = weights and biases
tf.reset_default_graph()

# placeholders for inputs and targets - what type and shape are they
inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.int32, [None, output_size])

# Shape of the weights matrix is: w = k * m, where k = number of input nodes, and m = number of hidden units
# Shape of the biases matrix is: b = 1 * m
weights_1 = tf.get_variable("weights_1",[input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])

# tf.nn is a module that contains neural network functions including activitation functions
# this line applies an activitation function to the linear combination of the inputs, weights and biases
outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

# Define the second hidden layer
weights_2 = tf.get_variable("weights_2", [hidden_layer_size,hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])

outputs_2 = tf.nn.relu(tf.matmul(outputs_1,weights_2) + biases_2)

#Define the output layer
weights_3 = tf.get_variable("weights_3", [hidden_layer_size,output_size])
biases_3 = tf.get_variable("biases_3", [output_size])

outputs = tf.matmul(outputs_2, weights_3) + biases_3

#logits are the unscaled probabilities. Softmax will create scaled probabilities - a vector summing to 1.
loss = tf.nn.softmax_cross_entropy_with_logits(logits = outputs, labels = targets)

#Mean loss gives a big performance boost
#reduce mean takes the mean of elements across a tensor
mean_loss = tf.reduce_mean(loss)

optimize = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(mean_loss)

out_equals_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)

#SGD batch size is 1. GD is # of samples. We want somewhere in between to get balance between speed and accuracy
batch_size = 100
max_epochs = 100
prev_validation_loss = 9999999.

train_data = Audiobooks_Data_Reader('train', batch_size)
validation_data = Audiobooks_Data_Reader('validation')

train_start_time = dt.now()

for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0.
    
    for input_batch, target_batch in train_data:
        _, batch_loss = sess.run([optimize, mean_loss],
                                feed_dict={inputs:input_batch, targets:target_batch})
        curr_epoch_loss += batch_loss
        
    curr_epoch_loss /= train_data.batch_count
    
    validation_loss = 0.
    validation_accuracy = 0.
    
    for input_batch, target_batch in validation_data:
        validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
                                feed_dict={inputs:input_batch, targets:target_batch})

    print('Epoch '+str(epoch_counter+1)+
        '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
        '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
        '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%'       
         )
    
    #Early stopping - if validation loss starts to increase - to prevent overfitting
    
    if validation_loss > prev_validation_loss:
        break
        
    prev_validation_loss = validation_loss

train_end_time = dt.now()
train_delta = train_end_time - train_start_time

print('End of training! Training time(s): '+'{0:.2f}'.format(train_delta.total_seconds()))

/home/radhey/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:1752: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 1. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 99.78%
Epoch 2. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 99.55%
Epoch 3. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 98.88%
Epoch 4. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 98.21%
Epoch 5. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 97.76%
Epoch 6. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 96.64%
Epoch 7. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 95.97%
Epoch 8. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 95.75%
Epoch 9. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 94.85%
Epoch 10. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 93.06%
Epoch 11. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 91.28%
Epoch 12. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 89.04%
E

Epoch 100. Training loss: 0.000. Validation loss: 0.000. Validation accuracy: 98.66%
End of training! Training time(s): 2.70


# Test the model

In [105]:

test_data = Audiobooks_Data_Reader('test')

for input_batch, target_batch in test_data:
    test_accuracy = sess.run([accuracy],
                        feed_dict={inputs: input_batch, targets: target_batch })

test_accuracy_percent = test_accuracy[0] * 100.

print('Test accuracy is: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy is: 97.54%


# Thank you